In [1]:
from train_utils import *

In [2]:
import jsonmemo as jsonmemo_module
#jsonmemo_funcs = jsonmemo_module.create_jsonmemo_funcs(getsecret('DATA_DUMP'), lowmem=True)
jsonmemo_funcs = jsonmemo_module.create_jsonmemo_funcs(getsecret('DATA_DUMP'))
jsonmemo1arg = jsonmemo_funcs['jsonmemo1arg']
jsonmemo = jsonmemo_funcs['jsonmemo']
mparrmemo = jsonmemo_funcs['mparrmemo']
msgpackmemo1arg = jsonmemo_funcs['msgpackmemo1arg']
msgpackmemo = jsonmemo_funcs['msgpackmemo']

In [3]:
# proposede costs of sampling

# time spent in answering the question (limit to instances where they do answers). how much time are we wasting for user each time they ask

# non-response rate. propobability of not responding within 30 seconds and going for the default radnom choice

# entropy over responses. window if they are in groups according to time spent on habitlab


In [4]:
print(len(get_users()))

4955


In [5]:
for user in get_users():
  install_logs = get_install_logs_for_user(user)

In [6]:
print((get_collection_items('collections'))[0])

{'_id': 'user_active_dates', 'userid': 'user', 'collection': 'active_dates'}


In [7]:
for x in get_collection_names():
  if 'nternal:choose_difficulty' in x:
    print(x)
    break

e0ea34c81d4b50cddc7bd752_internal:choose_difficulty


In [8]:
#a = get_all_features_data()

for user in get_users():
  user_to_difficulty_items = get_choose_difficulty_items_for_user(user)
  #if len(user_to_difficulty_items) > 0:
    #print(user)
    #break

In [9]:
difficulty_items = get_choose_difficulty_items_for_user('0a0e30452f272c10a9c2675d')
print(len(difficulty_items))

6


In [10]:
def get_choose_difficulty_items_for_user(user):
  collection_items = get_collection_for_user(user, 'internal:choose_difficulty')
  output = []
  for x in collection_items:
    if 'is_new_session' not in x:
      continue
    if x['is_new_session'] != True:
      continue
    if ('developer_mode' in x) and (x['developer_mode'] == True):
      continue
    if ('is_preview_mode' in x) and (x['is_preview_mode'] == True):
      continue
    if ('is_suggestion_mode' in x) and (x['is_suggestion_mode'] == True):
      continue
    if ('unofficial_version' in x):
      continue
    output.append(x)
  return output

#collection_items = get_collection_for_user('0a0e30452f272c10a9c2675d', 'internal:choose_difficulty')
#print((collection_items))
print(get_choose_difficulty_items_for_user('0a0e30452f272c10a9c2675d'))

[{'_id': ObjectId('5c6dd09ac325ff0015013c94'), 'url': 'https://www.youtube.com/watch?v=9DNVVBAwok4&feature=youtu.be', 'tab_id': 636, 'session_id': 0, 'is_new_session': True, 'is_preview_mode': False, 'is_suggestion_mode': False, 'is_previously_seen': False, 'type': 'impression', 'intervention': 'internal/choose_difficulty', 'userid': '0a0e30452f272c10a9c2675d', 'install_id': '101f52bf3bc5bdf4ea9d5008', 'day': 1147, 'synced': 0, 'timestamp': 1550700698228.0, 'localtime': 'Thu Feb 21 2019 05:11:39 GMT+0700 (Indochina Time)', 'itemid': '2895a47bbb345be468ec5660', 'log_major_ver': '8', 'log_minor_ver': '1', 'habitlab_version': '1.0.264', 'id': 1, 'logname': 'internal/choose_difficulty', 'timestamp_local': 1550700699193.0}, {'_id': ObjectId('5c6dd1c4c325ff001501510a'), 'url': 'https://www.youtube.com/watch?v=3nMVkycWN_w', 'tab_id': 647, 'session_id': 1, 'is_new_session': True, 'is_preview_mode': False, 'is_suggestion_mode': False, 'is_previously_seen': False, 'type': 'impression', 'interven

In [11]:
for x in difficulty_items:
  if 'is_random' not in x:
    continue
  print(x)
  break

{'_id': ObjectId('5c6dd1cec325ff00150151a9'), 'difficulty': 'easy', 'intervention_name': 'youtube/remove_comment_section', 'new_session': True, 'choose_difficulty_interface': 'this_intervention_toast', 'is_random': True, 'url': 'https://www.youtube.com/watch?v=3nMVkycWN_w', 'tab_id': 647, 'session_id': 1, 'is_new_session': True, 'is_preview_mode': False, 'is_suggestion_mode': False, 'is_previously_seen': True, 'type': 'action', 'intervention': 'internal/choose_difficulty', 'userid': '0a0e30452f272c10a9c2675d', 'install_id': '101f52bf3bc5bdf4ea9d5008', 'day': 1147, 'synced': 0, 'timestamp': 1550701006999.0, 'localtime': 'Thu Feb 21 2019 05:16:47 GMT+0700 (Indochina Time)', 'itemid': '27a81df1a845b259b3c8cdcf', 'log_major_ver': '8', 'log_minor_ver': '1', 'habitlab_version': '1.0.264', 'id': 3, 'logname': 'internal/choose_difficulty', 'timestamp_local': 1550701007468.0}


In [12]:
def get_non_response_rates_for_user(user):
  non_response = 0
  have_response = 0
  difficulty_items = get_choose_difficulty_items_for_user(user)
  for x in difficulty_items:
    is_random = x.get('is_random', None)
    if is_random == None:
      continue
    if is_random == True:
      non_response += 1
      continue
    if is_random == False:
      have_response += 1
      continue
  if (non_response + have_response) == 0:
    return None
  return non_response / (non_response + have_response)

#print(get_non_response_rates_for_user('8d2c9eb27dee2dc85bca705b'))

In [13]:
def set_dict(d, value, *dpath):
  for x in dpath[:-1]:
    if x not in d:
      d[x] = {}
    d = d[x]
  d[dpath[-1]] = value

def append_dict_list(d, value, *dpath):
  for x in dpath[:-1]:
    if x not in d:
      d[x] = {}
    d = d[x]
  if dpath[-1] not in d:
    d[dpath[-1]] = []
  d[dpath[-1]].append(value)

In [14]:
from browser_libs import get_user_to_all_install_ids

user_to_install_ids = get_user_to_all_install_ids()


In [15]:
unpaired_actions = 0
paired_actions = 0
unpaired_impressions = 0
paired_impressions = 0
impression_lengths = Counter()

@msgpackmemo1arg
def get_impressions_paired_with_actions(user):
  global paired_actions
  global unpaired_actions
  global paired_impressions
  global unpaired_impressions
  difficulty_items = get_choose_difficulty_items_for_user(user)
  output = []
  tab_id_to_session_id_to_impressions = {}
  tab_id_to_session_id_to_actions = {}
  #print(len(difficulty_items))
  for x in difficulty_items:
    tab_id = x['tab_id']
    session_id = x['session_id']
    if x['type'] == 'impression':
      append_dict_list(tab_id_to_session_id_to_impressions, x, tab_id, session_id)
    if x['type'] == 'action':
      append_dict_list(tab_id_to_session_id_to_actions, x, tab_id, session_id)
  #print(tab_id_to_session_id_to_impressions)
  for tab_id,session_id_to_actions in tab_id_to_session_id_to_actions.items():
    for session_id,actions in session_id_to_actions.items():
      if tab_id not in tab_id_to_session_id_to_impressions or (session_id not in tab_id_to_session_id_to_impressions[tab_id]):
        #print('missing impression for action:', tab_id, session_id)
        unpaired_actions += 1
      else:
        paired_actions += 1
  impressions_info_list = []
  actions_info_list = []
  for tab_id,session_id_to_actions in tab_id_to_session_id_to_actions.items():
    for session_id,actions in session_id_to_actions.items():
      is_paired = False
      if tab_id not in tab_id_to_session_id_to_impressions or (session_id not in tab_id_to_session_id_to_impressions[tab_id]):
        # action but no impression
        unpaired_actions += 1
        is_paired = False
      else:
        # action with corresponding impression
        paired_actions += 1
        is_paired = True
      if len(actions) != 1:
        continue
      action_info = copy(actions[0])
      action_info['is_paired'] = is_paired
      actions_info_list.append(action_info)
  for tab_id,session_id_to_impressions in tab_id_to_session_id_to_impressions.items():
    for session_id,impressions in session_id_to_impressions.items():
      is_paired = False
      if tab_id not in tab_id_to_session_id_to_actions or (session_id not in tab_id_to_session_id_to_actions[tab_id]):
        # impression but no selection
        unpaired_impressions += 1
        is_paired = False
      else:
        # impression with no corresponding action
        #print(impressions[0]['intervention'])
        paired_impressions += 1
        is_paired = True
      impression_lengths[len(impressions)] += 1
      if len(impressions) != 1:
        continue
      impression_info = copy(impressions[0])
      impression_info['is_paired'] = is_paired
      impressions_info_list.append(impression_info)
  actions_info_list.sort(key=lambda x: x['timestamp_local'])
  impressions_info_list.sort(key=lambda x: x['timestamp_local'])
  return impressions_info_list,actions_info_list,tab_id_to_session_id_to_impressions,tab_id_to_session_id_to_actions


for user in get_users():
  if user not in user_to_install_ids:
    continue
  if len(user_to_install_ids[user]) != 1:
    continue
  get_impressions_paired_with_actions(user)
#impressions_info_list = get_impressions_paired_with_actions('8d2c9eb27dee2dc85bca705b')
print('paired_actions', paired_actions)
print('unpaired_actions', unpaired_actions)
print('paired impressions', paired_impressions)
print('unpaired impressions', unpaired_impressions)
print('impression lengths')
print(impression_lengths)

paired_actions 0
unpaired_actions 0
paired impressions 0
unpaired impressions 0
impression lengths
Counter()


In [16]:
impressions_info_list,actions_info_list,tab_id_to_session_id_to_impressions,tab_id_to_session_id_to_actions = get_impressions_paired_with_actions('8d2c9eb27dee2dc85bca705b')

In [17]:
print(tab_id_to_session_id_to_impressions.keys())

dict_keys([279, 283, 287, 296, 307, 310, 15, 2, 38, 53, 57, 63, 67, 71, 80, 77, 92, 105, 111, 119, 133, 137, 143, 157, 231, 237, 243, 249, 261, 267, 273, 285, 316, 352, 385, 389, 394, 406, 412, 416, 420, 444, 448, 456, 464, 474, 490, 517, 496, 536, 555, 572, 578, 592, 598, 604, 610, 616, 639, 643, 649, 658, 664, 673, 706, 712, 720, 732, 746, 749, 756, 762, 786, 796, 802, 808, 820, 830, 836, 860, 872, 878, 884, 890, 913, 927, 933, 949, 955, 961, 973, 979, 13, 66, 81, 93, 136, 146, 152, 158, 183, 191, 197, 203, 255, 259, 265, 290, 301, 323, 369, 401, 405, 408, 413, 425, 429, 435, 468, 472, 476, 487, 491, 500, 504, 519, 524, 532, 544, 548, 559, 552, 564, 568, 584, 588, 618, 622, 626, 630, 634, 646, 650, 654, 662, 666, 678, 682, 686, 694, 698, 702, 709, 713, 717, 741, 753, 766, 773, 789, 797, 801, 809, 822, 825, 829, 841, 846, 855, 863, 866, 877, 899, 918, 940, 943, 964, 968, 988, 999, 1012, 1015, 1020, 1024, 1038, 1042, 1052, 1049, 1064, 1071, 1074, 1082, 1087, 1103, 1110, 1123, 1129, 114

In [18]:
latency_list = []
for x in actions_info_list:
  if not x['is_paired']:
    continue
  tab_id = x['tab_id']
  session_id = x['session_id']
  action_timestamp = x['timestamp_local']
  impression_info = tab_id_to_session_id_to_impressions[tab_id][session_id][0]
  impression_timestamp = impression_info['timestamp_local']
  latency = (action_timestamp - impression_timestamp) / 1000
  if latency > 30:
    continue
  latency_list.append(latency)


In [19]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

import numpy as np



In [20]:
#x = np.random.randn(500)
data = [go.Histogram(x=latency_list)]

iplot(data)

In [21]:
latency_list_all = []
for user in get_users():
  if user not in user_to_install_ids:
    continue
  if len(user_to_install_ids[user]) != 1:
    continue
  impressions_info_list,actions_info_list,tab_id_to_session_id_to_impressions,tab_id_to_session_id_to_actions = get_impressions_paired_with_actions(user)
  #get_impressions_paired_with_actions(user)
  for x in actions_info_list:
    if not x['is_paired']:
      continue
    tab_id = x['tab_id']
    session_id = x['session_id']
    action_timestamp = x['timestamp_local']
    impression_info = tab_id_to_session_id_to_impressions[tab_id][session_id][0]
    impression_timestamp = impression_info['timestamp_local']
    latency = (action_timestamp - impression_timestamp) / 1000
    if not (0 < latency < 30):
      continue
    latency_list_all.append(latency)


In [22]:
#x = np.random.randn(500)
data = [go.Histogram(x=latency_list_all)]

iplot(data)

In [23]:
prev_timestamp = 0
for x in impressions_info_list:
  print(x['is_paired'], x['install_id'], x['url'], x['tab_id'], x['session_id'], (x['timestamp_local'] - prev_timestamp) / 1000)
  prev_timestamp = x['timestamp_local']

True 1ea9c5090d98af02f55c8029 https://www.youtube.com/ 148 0 1555345357.06


In [24]:
def get_impression_times_prompted_for_user(user):
  difficulty_items = get_choose_difficulty_items_for_user(user)
  output = []
  for x in difficulty_items:
    #print(x['type'])
    print(x)
    impression_time = get_time_adjusted_for_timezone(x['timestamp'], x['localtime'])
    print(impression_time)
    if x['type'] == 'action': #'impression'
      break
    output.append(impression_time)
  return output

get_impression_times_prompted_for_user('8d2c9eb27dee2dc85bca705b')

{'_id': ObjectId('5c104667c6466e0014e3a47f'), 'url': 'https://www.facebook.com/', 'tab_id': 279, 'session_id': 0, 'is_new_session': True, 'is_preview_mode': False, 'is_suggestion_mode': False, 'is_previously_seen': False, 'type': 'impression', 'intervention': 'internal/choose_difficulty', 'userid': '8d2c9eb27dee2dc85bca705b', 'install_id': '70870c04da0d5fee89b69c40', 'day': 1075, 'synced': 0, 'timestamp': 1544570471554.0, 'localtime': 'Tue Dec 11 2018 23:57:49 GMT+0100 (Central European Standard Time)', 'itemid': 'a8b4d51da34a71ebb1b33824', 'log_major_ver': '8', 'log_minor_ver': '1', 'habitlab_version': '1.0.258', 'id': 1, 'logname': 'internal/choose_difficulty', 'timestamp_local': 1544569069091.0}
2018-12-12T00:21:11.554000+00:00
{'_id': ObjectId('5c10466713990a0014b5b7c7'), 'difficulty': 'medium', 'intervention_name': 'facebook/show_user_info_banner', 'new_session': True, 'url': 'https://www.facebook.com/', 'tab_id': 279, 'session_id': 0, 'is_new_session': True, 'is_preview_mode': Fa

[<Arrow [2018-12-12T00:21:11.554000+00:00]>]

In [25]:
def get_response_latencies_for_user(user):
  difficulty_items = get_choose_difficulty_items_for_user(user)
  for x in difficulty_items:
    is_random = x.get('is_random', None)
    if is_random == False:
      pass
      # TODO finish writing

In [26]:
for user in get_users():
  response_rate = get_non_response_rates_for_user(user)
  if response_rate == None:
    continue
  print(response_rate)

0.47613458528951486
0.5961538461538461
0.5714285714285714
0.925
0.5494505494505495
0.7159090909090909
0.6709401709401709
0.48484848484848486
0.625
0.5245901639344263
1.0
0.5555555555555556
0.7049608355091384
0.5321100917431193
0.5280373831775701
0.5603448275862069
0.5714285714285714
0.5
0.5
0.575
0.532258064516129
0.5
0.6
0.4801980198019802
0.75
0.5595238095238095
0.5238095238095238
0.5
0.5172413793103449
1.0
0.680672268907563
1.0
0.5588235294117647
0.6923076923076923
0.5
0.5
0.45652173913043476
0.5188470066518847
0.55
0.6477272727272727
0.712682379349046
0.7333333333333333
0.6243523316062176
0.5
0.6
0.5189873417721519
0.5
0.8867924528301887
1.0
0.5
1.0
0.6666666666666666
0.5
0.5588235294117647
0.5
0.5
0.66
1.0
0.9565217391304348
0.6538461538461539
0.6025641025641025
0.5416666666666666
0.5
0.5
0.7843137254901961
0.6923076923076923
0.5
0.5
0.9397590361445783
0.5384615384615384
0.5848101265822785
0.550561797752809
0.5643564356435643
0.5570776255707762
0.6837606837606838
0.852760736196319

0.8493150684931506
0.5958549222797928
0.5909090909090909
0.7596153846153846
0.5681818181818182
0.9921671018276762
0.6666666666666666
0.6476683937823834
0.6097560975609756
0.4
0.5925925925925926
0.5833333333333334
0.7384615384615385
0.8076923076923077
0.8666666666666667
0.5
0.0
0.6666666666666666
0.5697674418604651
0.8333333333333334
1.0
0.5
0.7045454545454546
1.0
1.0
0.6
0.7142857142857143
0.6785714285714286
0.6111111111111112
1.0
0.8333333333333334
0.5
0.5087719298245614
0.7416666666666667
0.5
0.5405405405405406
0.8571428571428571
0.5257142857142857
0.52
1.0
0.5
0.5498866213151927
0.6607142857142857
0.5112359550561798
0.5714285714285714
1.0
0.8805970149253731
0.55
1.0
0.5671641791044776
0.56
0.5109983079526227
0.5163934426229508
1.0
0.75
0.8260869565217391
0.5666666666666667
0.624113475177305
0.5
0.6521739130434783
0.6923076923076923
0.5263157894736842
0.6527777777777778
0.7140255009107468
1.0
0.6818181818181818
0.5984251968503937
0.8823529411764706
0.5777777777777777
1.0
0.6129032258

0.6666666666666666
0.5955056179775281
1.0
0.5666666666666667
0.75
0.5733333333333334
1.0
0.624
0.8181818181818182
0.7228915662650602
0.8571428571428571
0.6666666666666666
1.0
1.0
0.5562913907284768
0.6
0.72
1.0
0.6666666666666666
1.0
0.7459016393442623
0.5
0.5123456790123457
0.5925925925925926
0.5555555555555556
0.5
0.5
0.6428571428571429
0.5915492957746479
0.5
0.6556603773584906
0.8571428571428571
0.6486486486486487
0.5
0.7692307692307693
1.0
0.8571428571428571
0.5650557620817844
0.654639175257732
0.6
1.0
0.582089552238806
0.5851851851851851
0.8
1.0
1.0
0.8333333333333334
0.47607655502392343
0.28378378378378377
1.0
0.0
0.53125
0.6346153846153846
0.5641025641025641
1.0
0.65625
0.9666666666666667
0.6551724137931034
0.782608695652174
1.0
0.5
0.6082949308755761
0.5777777777777777
0.5472972972972973
1.0
0.75
1.0
0.5454545454545454
0.6296296296296297
0.58
0.6818181818181818
0.75
1.0
0.6444444444444445
1.0
0.75
0.6666666666666666
0.8888888888888888
0.8571428571428571
0.5714285714285714
0.594

0.5754385964912281
0.5714285714285714
0.3595505617977528
0.47368421052631576
1.0
0.6363636363636364
0.9285714285714286
0.5833333333333334
0.5416666666666666
0.4842105263157895
0.5
0.5
0.5072463768115942
0.7272727272727273
0.49244712990936557
0.6132596685082873
0.8
1.0
0.8125
0.75
0.7659574468085106
0.6774193548387096
0.8
0.5360824742268041
1.0
0.5686274509803921
0.7066666666666667
1.0
0.5964912280701754
1.0
1.0
1.0
0.6
0.5698924731182796
0.5
0.8
0.8260869565217391
1.0
0.5813953488372093
0.625
0.9938837920489296
1.0
0.875
0.547486033519553
0.5
0.5166666666666667
0.6666666666666666
0.5517241379310345
0.6428571428571429
0.45
1.0
0.5238095238095238
0.6442307692307693
1.0
0.5539568345323741
0.8333333333333334
0.726027397260274
0.26666666666666666
1.0
1.0
0.5454545454545454
0.6166666666666667
0.7333333333333333
0.7
1.0
0.8888888888888888
0.5
1.0
1.0
0.6195121951219512
0.6388888888888888
0.49264705882352944
0.8
0.5833333333333334
0.6153846153846154
0.5555555555555556
0.6369426751592356
0.5
0.

1.0
0.75
0.75
1.0
0.6666666666666666
0.4857142857142857
0.5
0.5
0.5
0.7592592592592593
0.5714285714285714
0.8622129436325678
0.875
0.5
0.8102189781021898
0.6666666666666666
0.5897435897435898
0.8545454545454545
1.0
0.6470588235294118
0.7142857142857143
0.8823529411764706
0.5932203389830508
0.5333333333333333
0.6176470588235294
0.7636363636363637
0.6764705882352942
0.5319148936170213
0.9776119402985075
0.5357142857142857
0.375
0.5
0.7
0.7254901960784313
0.7333333333333333
1.0
0.75
1.0
0.9
0.4943181818181818
1.0
0.75
0.9
0.5
0.7239263803680982
1.0
0.5
0.5142857142857142
0.6724137931034483
0.5909090909090909
0.75
0.5
0.7241379310344828
1.0
0.6153846153846154
0.5
0.6273291925465838
0.6
1.0
0.4939759036144578
0.7272727272727273
0.9285714285714286
0.9072164948453608
0.5327868852459017
0.625
0.6511627906976745
0.5714285714285714
0.6153846153846154
0.8947368421052632
0.7548387096774194
0.5
1.0
0.3333333333333333
0.5238095238095238
0.48
0.6
1.0
0.6125
0.7142857142857143
0.896551724137931
0.8181

0.6052631578947368
1.0
0.5272727272727272
0.6721311475409836
0.78125
1.0
0.5
0.625
0.5454545454545454
0.6923076923076923
0.5
0.8888888888888888
0.52
1.0
0.8
0.5
1.0
1.0
0.75
0.6666666666666666
0.6666666666666666
1.0
0.7142857142857143
1.0
0.5135135135135135
0.6981132075471698
1.0
0.6666666666666666
0.5625
0.6666666666666666
1.0
0.6666666666666666
0.5625
0.8306451612903226
0.5833333333333334
0.5833333333333334
0.5526315789473685
0.5
0.4908350305498982
0.5119047619047619
1.0
1.0
1.0
1.0
1.0
0.6515151515151515
1.0
0.5454545454545454
0.75
0.5
1.0
0.6666666666666666
0.7777777777777778
1.0
0.8461538461538461
0.5
1.0
0.5
0.5434782608695652
1.0
1.0
0.0
0.5
0.5
0.5
0.6666666666666666
0.9782608695652174
0.7
0.8426966292134831
1.0
1.0
0.6
0.5882352941176471
0.6086956521739131
0.5833333333333334
0.6285714285714286
1.0
0.5
0.7272727272727273
1.0
1.0
0.7894736842105263
0.6666666666666666
1.0
1.0
0.8367346938775511
0.8279569892473119
1.0
1.0
0.75
1.0
1.0
0.9090909090909091
0.5092592592592593
0.533333